### Defined Functions

In [1]:
def weekly_sched_extract(username, password):
    # schedule extractor
    import re
    import time
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from bs4 import BeautifulSoup
    import pandas as pd
    
    driver = webdriver.Chrome()
    link = r"https://aisis.ateneo.edu/j_aisis/J_VMCS.do"
    
    driver.get(link)
    time.sleep(1)
    
    driver.find_element(By.NAME, 'userName').send_keys(username) 
    driver.find_element(By.NAME, 'password').send_keys(password)
    driver.find_element(By.NAME, "submit").click()
    time.sleep(1)
    
    driver.find_element(By.XPATH, '//a[contains(@href, "J_VMCS.do")]').click()
    time.sleep(1.5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    
    # find the table with the schedule
    all_tables = soup.find_all("table")
    schedule_table = None
    for table in all_tables:
        # check for presence of headers or specific keywords in table cells
        if any("Time" in cell.text.strip() for cell in table.find_all("td")[:2]):
            schedule_table = table
            break
    print('Schedule Found! Please wait for processing...')
    
    # extract data from the table and load it into dataframe
    rows = schedule_table.find_all("tr") # all rows are indicated by "tr" when printing the extracted soup
    data = [] # initializing
    headers = [header.text.strip() for header in rows[0].find_all("td")]
    # rows[0] accesses the first row, which for this soup object contains all the table headers 'Time', 'Mon, 'Tue', 'Wed', etc.
    # find_all("td") finds all the column elements inside this header row
    # header.text.strip() extracts the text, removes any whitespace, and stores the text it extracts as headers for the df
    
    for row in rows[1:]:
        # we skip the header row since it's already been skimmed over
        cols = row.find_all("td")
        # finding all the column elements across all rows barring the header row
        data.append([col.text.strip() for col in cols])
    
    sched_df = pd.DataFrame(data, columns=headers)
    
    # panda extractor
    def panda_schedule_extractor(df):
        schedules = []
        days = df.columns[1:]  # excluding the column for 'Time' to store the names of the days
    
        for day in days:
            current_class = None # initializing current_class and start_time as None
            start_time = None
            for index, row in df.iterrows(): # iterates over each row in the DataFrame
                time = row['Time'] # stores the current time range from column 'Time'
                class_name = row[day] # stores the class name for the current 'day'
    
                if class_name and class_name != current_class: # if class_name exists (not empty) and it's different from the current_class, the program reads it as a new class starting
                    if current_class: # if there's an ongoing class, it still occupies the current time slot
                        end_time = df.at[index-1, 'Time'].split('-')[1] #the end time of the current class is the end time of the previous time slot (hence index-1) 
                        schedules.append(f"{current_class} {current_day} {start_time}-{end_time}") #all this is appended to the schedule string here
                    current_class = class_name # the current_class is updated to the new name of the new class
                    current_day = day # current_day is updated to the day of the new class
                    start_time = time.split('-')[0] # start time is set to the start time of the current time_slot
                elif class_name != current_class: # this elif-if checks if an ongoing class is about to end
                    if current_class: # if there's an ongoing class and class_name 
                        end_time = df.at[index-1, 'Time'].split('-')[1] 
                        schedules.append(f"{current_class} {current_day} {start_time}-{end_time}")
                    current_class = None
    
            # Handle the last ongoing class of the day
            if current_class:
                end_time = time.split('-')[1] # end time of the last ongoing class is the end time of the current time slot
                schedules.append(f"{current_class} {current_day} {start_time}-{end_time}")
    
        return schedules
    
    # Extract schedules
    class_schedules = panda_schedule_extractor(sched_df)
    
    #print(class_schedules)
    
    final_sched_df = pd.DataFrame(columns = ['Subject', 'Location', 'Day', 'Start Time', 'End Time'])
       
    def parse_schedule(schedule):
        # Split by spaces and parentheses
        parts = item.split()
        
        # Extract subject and location
        subject = ' '.join(parts[0:2])
        location = ' '.join(parts[2:-2])
        
        # Extract day and time
        day = parts[-2]
        start_time_str, end_time_str = parts[-1].split('-')
        
        # Format times
        start_time = f"{start_time_str[:-2]}:{start_time_str[-2:]}"
        end_time = f"{end_time_str[:-2]}:{end_time_str[-2:]}"
    
        temp_list = [subject, location, day, start_time, end_time]
    
        return temp_list
    
    for item in class_schedules:
        final_sched_df.loc[len(final_sched_df.index)] = parse_schedule(item)
    
    def section_specify(subject):
        section = subject[-1] 
        final_subject = subject[0:-1]
        final_subject += f' Section {section}'
        return final_subject
    
    final_sched_df['Subject'] = final_sched_df['Subject'].apply(section_specify)

    print('Uploading Table to Google Spreadsheets Now!')
    
    return final_sched_df

'================================================='

def word_list_creation(text):
    # Transformation + Removal of Unwanted Punctuations
    text = text.upper()
    text = text.replace("\n"," ")
    
    punctuations = r"&$@[].,'#()-\"!?’_;:/●"
    text = text.replace('"', " ")
    for symbol in punctuations:
        text = text.replace(symbol, " ")
    
    word_list = text.split()
    
    return word_list

"=================================================="

def keyword_find(text):
    import spacy
    from nltk.corpus import wordnet as wn
    from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
    
    # Loading spaCy model with word vectors, disabling unnecessary components 'parser', 'ner', and 'textcat' for max efficiency
    nlp = spacy.load('en_core_web_md', disable=['parser', 'ner', 'textcat'])
    
    # Given seed terms
    seed_terms = ["TEST", "EXAM", "PAPER", "ESSAY", "PROJECT", "PODCAST", "CASE", "ASSESSMENT", "PROBLEM SET", "NEWSLETTER", "ANALYSIS", "ASSIGNMENT", "SUMMATIVE"]
    
    # Custom stop words - these are words we want the program to skip over, no need to analyze these since they're unrelated to our seed terms
    custom_stop_words = set([
        'course', 'professor', 'syllabus', 'university', 'student', 'class', 
        'week', 'lecture', 'grade', 'school', 'core', 'curriculum', 'outcomes', 'cclo', 'critical', 'program', 'plo', 'outline', 'hours',
        'module', 'print', 'online', 'onsite', 'instructor/s', 'information', 'year', 'department', 'instructor', 
    ])
    
    keyword = "LEARNING METHODS"
    def preprocess_syllabus(syllabus_text, keyword):
        # remove all text that come after "LEARNING METHODS"
        def remove_text_after_string(text, keyword):
            keyword_upper = keyword.upper()
            index = text.find(keyword_upper)
            if index != -1:
                return text[:index]
            return text
    
        # Remove text after the specified keyword
        syllabus_text = remove_text_after_string(syllabus_text, keyword)
    
        # transformations to remove unwanted punctuations and convert to upper case
        punctuations = r"&$@[].,'#()-\"!?’_;:/●"
        syllabus_text = syllabus_text.upper().replace("\n", " ").replace('"', " ")
        for symbol in punctuations:
            syllabus_text = syllabus_text.replace(symbol, " ")
    
        # tokenize, remove stopwords, and filter by part of speech (POS) tags, mainly NOUN, VERB, ADJ (adjective), and ADV (adverb)
        doc = nlp(syllabus_text)
        tokens = [
            token.text for token in doc 
            if token.text.lower() not in ENGLISH_STOP_WORDS # this refers to spaCy's own database of common stop words, such as "and", "or", "the", etc.
            and token.text.lower() not in custom_stop_words # this is based on our stop words
            and token.pos_ in {"NOUN"} # checks if the word is tagged as a noun
        ]
        return tokens # if all conditions are met they're included in the list of words to be analyzed
    
    def calculate_similarity_wordnet(word, seed_terms, cache):
        if word in cache: # super important to use caches to avoid redundant calculations
            # function first checks if the word it's analyzing has already been analyzed for similarity
            return cache[word] # if the word has already been analyzed, function immediately returns the cached value (this saves so much time)
        
        max_similarity = 0 # initialized to 0
        word_synsets = wn.synsets(word) # retrieves all synsets (synonyms) for the current word being analyzed; synonyms lifted from WordNet database
        for term in seed_terms:
            term_synsets = wn.synsets(term) # retrieves all WordNet synonyms for each seed term in the seed_term list
            for ws in word_synsets:
                for ts in term_synsets:
                    similarity = wn.wup_similarity(ws, ts) # compares the similarity between the synsets of one word with the synsets of one seed_term
                    if similarity and similarity > max_similarity: # if the similarity is significant, it's recorded as the max_similarity
                        max_similarity = similarity 
        cache[word] = max_similarity
        return max_similarity
    
    def calculate_similarity_vectors(words, seed_terms, nlp, cache): # same logic as the previous function, except this time, it's based on vectors
        words_to_process = [word for word in words if word not in cache]
        word_docs = nlp(" ".join(words_to_process))
        term_docs = nlp(" ".join(seed_terms)) # being very honest i have 0 idea how this works, i just read the documentation for it, and with my trusty 
        # steed chatgpt I cooked this up and it works
        # supposedly this is used in conjunction with WN-similarity for more accurate results: when I tried to use only one without the other, the results
        # were indeed less accurate and significantly far from the seed terms
        
        for word_doc in word_docs:
            if not word_doc.has_vector:
                cache[word_doc.text] = 0  # Assign a default similarity of 0 for words without vectors
                continue  
            max_similarity = 0
            for term_doc in term_docs:
                if not term_doc.has_vector:
                    continue  
                similarity = word_doc.similarity(term_doc)
                if similarity > max_similarity:
                    max_similarity = similarity
                    
            cache[word_doc.text] = max_similarity
            
        return [cache[word] for word in words if word in cache]

    def extract_keywords(syllabus_text, num_keywords=10):
        # Preprocess the syllabus text
        preprocessed_tokens = preprocess_syllabus(syllabus_text, keyword)
        
        wordnet_cache = {}
        vector_cache = {}
        
        # Calculate combined similarity for each token
        token_similarities = {}
        wordnet_similarities = [calculate_similarity_wordnet(token, seed_terms, wordnet_cache) for token in preprocessed_tokens]
        vector_similarities = calculate_similarity_vectors(preprocessed_tokens, seed_terms, nlp, vector_cache)
        
        for token, wn_sim, vec_sim in zip(preprocessed_tokens, wordnet_similarities, vector_similarities):
            combined_similarity = (wn_sim + vec_sim) / 2  # Average of both similarities
            token_similarities[token] = combined_similarity
        
        # Sort tokens by combined similarity
        sorted_tokens = sorted(token_similarities.items(), key=lambda item: item[1], reverse=True)
        
        # Extract top keywords based on combined similarity
        keywords = [token for token, sim in sorted_tokens[:num_keywords]]
        
        return keywords
    
    keywords = extract_keywords(text)
    seed_copy = seed_terms.copy()
    final_keywords_set = list(set(seed_copy))
    
    for i in keywords:
        if i not in final_keywords_set:
            final_keywords_set.append(i)
    return final_keywords_set

"=================================================="

def date_find(word_list, text):
    # DATA CLEANER
    # Process 1: finds the general area of the assessments in the string
    keywords_list = keyword_find(text)
    
    process1_list = []
    for word in range(len(word_list)):
        if word_list[word] in keywords_list:
            process1_list.append(word_list[word-1:word+15]) # the reason why i-1 is because things like TEST are preceeded by things like LONG or SUMMATIVE
        
    
    # Process 2: finds the one with dates in them, the ones with a set deadline
    months_list = ["JAN","JANUARY","FEB","FEBRUARY","MAR","MARCH","APR","APRIL","MAY","JUN","JUNE","JUL","JULY","AUG","AUGUST","SEP","SEPT", "SEPTEMBER", "OCT","OCTOBER","NOV","NOVEMBER","DEC","DECEMBER"]
    
    process2_list = []
    for process1_sublists in process1_list:
        for process1_sublist_elements in process1_sublists:
            if process1_sublist_elements in months_list: # checks if one of the elements in the process1_sublists has the one of the months listed in months_list
                process2_list.append(process1_sublists)
    
    # Process 3: get some description followed by dates
    process3_list = []
    for process2_sublists in process2_list:
        for element in range(len(process2_sublists)):
            if process2_sublists[element].isdigit() == True and element > 2: # j > 2 to account for things like LONG TEST 2, SUMMATIVE EXAM 1
                if process2_sublists[0:element+1][-2] in months_list:
                    process3_list.append(process2_sublists[0:element+1]) # appends description with the digit of the date as the last item of that list (month-day format)
                else:
                    process3_list.append(process2_sublists[0:element+2]) # same thing as above but (day-month format)
                    
    
    # Process 4: separate the descriptions list from the dates list
    process4_list = []
    for process3_sublists in process3_list:
        day_month_fixer = [] # resets the list; important for the cases when multiple dates are formatted as day-month
        process4_list.append(process3_sublists[0:-2]) # description separator
        if process3_sublists[-2:len(process3_sublists)][-2] in months_list: # for month-day cases
            process4_list.append(process3_sublists[-2:len(process3_sublists)])
        else:
            # converts the day-month into month-day
            day_month_fixer.append(process3_sublists[-2:len(process3_sublists)][-1])
            day_month_fixer.append(process3_sublists[-2:len(process3_sublists)][-2])
            process4_list.append(day_month_fixer)
    
    # Process 5: convert both the descriptions and the dates list into strings
    process5_list = []
    for process4_sublists in process4_list:
        processed_string = ""
        for every_word in process4_sublists:
            processed_string += every_word + " "
        process5_list.append(processed_string.rstrip()) # rstrip to clean the space at the end
    
    # Process 6: final catcher to make sure that the key-value in the dictionary are description-date
    process6_list = []
    k = 1
    while k <= len(process5_list):
        if process5_list[k][0:process5_list[k].find(" ")] in months_list: # print process5_list, there are some shady values that get through, these clears any non-month values
            process6_list.append(process5_list[k-1])
            process6_list.append(process5_list[k])
        k += 2
    
    # Process 7: return a dictionary with the description as the key and the dates as the value
    final_dict = {}
    for l in range(0, len(process6_list),2):
        final_dict[process6_list[l]] = process6_list[l+1]
    
    # Data Cleaning
    remove_list = []
    key_list = list(final_dict.keys())
    for key in range(len(final_dict)):
        if key_list[key] in key_list[key-1]:
            remove_list.append(key_list[key])        
    
    for item in remove_list:
        final_dict.pop(item)

    return final_dict

"=================================================="

def panda_creation(dict, word_list, df):
    import pandas as pd

    def find_words_index(word_list, word1, word2):
        for i in range(len(word_list) - 1):
            if word_list[i].lower() == word1.lower() and word_list[i + 1].lower() == word2.lower():
                return i+1
                break
    
    def title_find(word_list):
        num1 = find_words_index(word_list, 'COURSE', 'NUMBER')
        num2 = find_words_index(word_list, 'NO', 'OF')
    
        if type(num1) == int and type(num2) == int:
            title_list = word_list[num1+1:num2-1]
            
            if len(title_list) == 3:
                subject = title_list[0] + ' ' + title_list[1] + '.' + title_list[2]
            else:
                subject = ' '.join(title_list)
            return subject
        return "MISSING SUBJECT"
    
    def location_find(word_list):
        
        if 'VENUE' in word_list and 'SECTION' in word_list:
            num3 = word_list.index('VENUE')
            num4 = word_list.index('SECTION')
            location_list = word_list[num3 + 1: num4]
    
            if len(location_list) > 8 or len(location_list) < 1:
                location = 'MISSING INFORMATION'
            else:
                location = ' '.join(location_list)
            return location
        return "MISSING LOCATION"
        
    title = title_find(word_list)
    location = location_find(word_list)
    
    for key in dict:
        temp_row = [title, key, location, dict[key]]
        df.loc[len(df.index)] = temp_row

'===================================================='

def create_events():
    import os.path  # to handle file paths
    from datetime import datetime as dt # for handling dates and times
    import pytz  # Import the pytz library for time zone management
    
    # Import necessary modules for Google authentication and API access
    from google.auth.transport.requests import Request  # For handling HTTP requests during authentication
    from google.oauth2.credentials import Credentials  # For managing OAuth 2.0 credentials
    from google_auth_oauthlib.flow import InstalledAppFlow  # For handling OAuth 2.0 flow for installed apps
    from googleapiclient.discovery import build  # For creating a service object to access the Google API
    from googleapiclient.errors import HttpError  # For handling HTTP errors from the API
    
    from datetime import datetime
    
    def parse_date(date_str):
        # Define the year you want to use
        year = datetime.now().year
    
        # Parse the date string "JULY 13" to "YYYY-MM-DD"
        date_obj = datetime.strptime(f"{year} {date_str}", "%Y %B %d")
        return date_obj.date().isoformat()
        
    Scopes = ['https://www.googleapis.com/auth/calendar']
    
    
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', Scopes)
            creds = flow.run_local_server(port = 0)
    
        with open('token.json','w') as  token:
            token.write(creds.to_json())
        
    service = build("calendar",'v3', credentials = creds)
    manila_tz = pytz.timezone('Asia/Manila')
        
    googleSheetId = '1m8badixRPCpTt-dKThNeqW98PK4naJpHsAoUKA_0Uro'
    workSheetName = 'Syllabus_Dates'
    URL = 'https://docs.google.com/spreadsheets/d/{0}/gviz/tq?tqx=out:csv&sheet={1}'.format(googleSheetId, workSheetName)
    date_df = pd.read_csv(URL)
        
    if date_df.empty:
        print("No data found in Syllabus_Dates sheet.")
    else:
        date_df['Summary'] = date_df.apply(lambda x: x.Subject + ' - ' + x.Summary, axis=1)
        del date_df['Subject']
        date_df['Description'] = date_df['Description'].apply(lambda x: '' if pd.isna(x) else x)
        date_df['Color ID'] = date_df['Color ID'].apply(lambda x: 0 if pd.isna(x) else int(x.split(' - ')[0]))
        
        for index, row in date_df.iterrows():  
            event = {
                'summary': row['Summary'],
                'location': row['Location'],
                'description': row['Description'],
                'colorId': row['Color ID'],
                'start': {
                    'date': parse_date(row['Date']),
                    'timeZone': 'Asia/Manila'
                },
                'end': {
                    'date': parse_date(row['Date']),
                    'timeZone': 'Asia/Manila'
                },
                'recurrence': ['RRULE:FREQ=DAILY;COUNT=1'],
            }
            event = service.events().insert(calendarId='primary', body=event).execute()
            print(f"Event {index + 1} of {len(date_df)} created: {row['Summary']}")
    
    print('')
    
    '===================================================='
    
    googleSheetId2 = '1HIzSiip1TT7wwGRrLnxLwEnqickJrbc-EunKEmY-z_k'
    workSheetName2 = 'Weekly_Schedule'
    URL2 = 'https://docs.google.com/spreadsheets/d/{0}/gviz/tq?tqx=out:csv&sheet={1}'.format(googleSheetId2, workSheetName2)
    schedule_df = pd.read_csv(URL2)
    
    if schedule_df.empty:
        print("No data found in Weekly_Schedule sheet.")
    else:
        schedule_df['Color ID'] = schedule_df['Color ID'].apply(lambda x: 0 if pd.isna(x) else int(x.split(' - ')[0]))
        schedule_df['Day'] = schedule_df['Day'].apply(lambda x: x[0:2].upper())
        
        def parse_datetime(year, date_str, time_str):
            # Combine the provided date string with the time string to create a datetime object
            datetime_obj = dt.strptime(f"{year} {date_str} {time_str}", "%Y %B %d %H:%M")
            return datetime_obj.isoformat()
    
        def create_weekly_event(row, current_year):
            start_datetime = parse_datetime(current_year, row['Start of Classes'], row['Start Time'])
            end_datetime = parse_datetime(current_year, row['Start of Classes'], row['End Time'])
            
            # Convert end_date_str to the UNTIL format
            end_date_obj = dt.strptime(row['End of Classes'], "%B %d")
            until_date = end_date_obj.replace(year=current_year).strftime("%Y%m%dT235959Z")
            
            byday = row['Day']
            
            event = {
                'summary': row['Subject'],
                'location': row['Location'],
                'description': row['Description'],
                'colorId': row['Color ID'],  
                'start': {
                    'dateTime': start_datetime,
                    'timeZone': 'Asia/Manila'
                },
                'end': {
                    'dateTime': end_datetime,
                    'timeZone': 'Asia/Manila'
                },
                'recurrence': [f'RRULE:FREQ=WEEKLY;BYDAY={byday};UNTIL={until_date}'],
            }
            return event
    
        current_year = dt.now().year
        
        for index, row in schedule_df.iterrows():
            event = create_weekly_event(row, current_year)
            event = service.events().insert(calendarId='primary', body=event).execute()
            print(f"Class {index + 1} of {len(schedule_df)} uploaded: {row['Subject']}")
            
    print('\nEverything is finished uploading! Please check out your calendar!\n')

def sheets_main(df, spreadsheet, sheet):
    import os

    # Import necessary modules for Google authentication and API access
    from google.auth.transport.requests import Request  # For handling HTTP requests during authentication
    from google.oauth2.credentials import Credentials  # For managing OAuth 2.0 credentials
    from google_auth_oauthlib.flow import InstalledAppFlow  # For handling OAuth 2.0 flow for installed apps
    from googleapiclient.discovery import build  # For creating a service object to access the Google API
    from googleapiclient.errors import HttpError  # For handling HTTP errors from the API
    
    sheets_scopes = ['https://www.googleapis.com/auth/spreadsheets']
        
    creds = None  # Initialize credentials to None

    # Check if 'token.json' exists in the current directory
    if os.path.exists('sheets_token.json'):
        # Load credentials from 'token.json' if it exists
        creds = Credentials.from_authorized_user_file('sheets_token.json')

    # Check if credentials are invalid or missing
    if not creds or not creds.valid:
        # If credentials exist but are expired and have a refresh token, refresh them
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # If no valid credentials, start the OAuth flow to get new credentials
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', sheets_scopes)
            # Run the local server for OAuth flow and get the credentials
            creds = flow.run_local_server(port=0)

        # Save the newly acquired credentials to 'token.json' for future use
        with open('sheets_token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials = creds)
    sheets = service.spreadsheets()

    # Convert the DataFrame to a list of lists
    values = [df.columns.tolist()] + df.values.tolist()

    # Define the body of the request
    body = {
        'values': values
    }

    # Update the sheet with the DataFrame values
    result = sheets.values().update(
        spreadsheetId=spreadsheet,
        range=f'{sheet}!B1',  
        valueInputOption='USER_ENTERED',
        body=body
        ).execute()

    print(f"{result.get('updatedCells')} cells updated in the sheet titled '{sheet}' found in https://docs.google.com/spreadsheets/d/{spreadsheet}!")

'==================================='

def get_n_events(number):
    import os.path  # to handle file paths
    import datetime as dt  # for handling dates and times
    import pytz  # Import the pytz library for time zone management
    
    # Import necessary modules for Google authentication and API access
    from google.auth.transport.requests import Request  # For handling HTTP requests during authentication
    from google.oauth2.credentials import Credentials  # For managing OAuth 2.0 credentials
    from google_auth_oauthlib.flow import InstalledAppFlow  # For handling OAuth 2.0 flow for installed apps
    from googleapiclient.discovery import build  # For creating a service object to access the Google API
    from googleapiclient.errors import HttpError  # For handling HTTP errors from the API
    
    Scopes = ['https://www.googleapis.com/auth/calendar']
    
    
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', Scopes)
            creds = flow.run_local_server(port = 0)
    
        with open('token.json','w') as  token:
            token.write(creds.to_json())
        
    service = build("calendar",'v3', credentials = creds)
    philippine_tz = pytz.timezone('Asia/Manila')  # Define the Philippine time zone
    now = dt.datetime.now(philippine_tz).isoformat()  # Get the current time in Philippine time zone
          
    event_result = service.events().list(calendarId = 'primary', timeMin = now, maxResults = number, singleEvents = True, orderBy = 'startTime').execute()
    
    events = event_result.get('items', [])
    
    if not events:
        print("No upcoming events found!")
        return

    i = 1
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        end = event['end'].get('dateTime', event['end'].get('date'))
        
        if 'T' in start:
            start_dt = dt.datetime.fromisoformat(start)
            start = start_dt.strftime('%Y-%m-%d %H:%M:%S')
        if 'T' in end:
            end_dt = dt.datetime.fromisoformat(end)
            end = end_dt.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Event # {i}: {event['summary']} | Starts on: {start} | Ends on: {end} |")
        i += 1


### User Interface

In [ ]:
from tkinter import * # tkinter imports
from tkinter import font
from tkinter import messagebox
from datetime import date

import webbrowser # image manipulation imports
import subprocess
import pyautogui
from PIL import Image, ImageEnhance

import os

import requests
import time
import threading
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import re
import time  
import fitz

# x------------------------------------------------------------------------------------------------------------------------------------x #
# Window Properties
tk_root = Tk()
tk_root.title("Calendar Organizer Dashboard")
tk_root.configure(bg="#402E7A")
tk_root.geometry("900x600") # window size can be changed to what looks aesthetically pleasing
tk_root.resizable(False, False)

# x------------------------------------------------------------------------------------------------------------------------------------x #
def main_page():

    # x------------------------------------------------------------------------------------------------------------------------------------x #
    # everything under the def date_extraction_page contains the code for the different extraction functions
    def date_extraction_page():
        main_Frame.destroy()

        # this function is what the "BACK" button in the date_extraction_page executes
        def date_extraction_to_main_back():
            date_extraction_Frame.destroy()
            main_page()

        # x------------------------------------------------------------------------------------------------------------------------------------x #
        # everything under the def aisis_credentials_class_schedule_page is used to generate the credentials page for the CLASS SCHEDULE SCRAPER
        def aisis_credentials_class_schedule_page():
            
            # removes the container of the date_extraction page, to be replaced by the aisis credentials
            date_extraction_Frame.destroy()
            
            # when called, destroys the container of aisis_credentials_class_schedule_page and recalls the date_extraction_page
            def aisis_to_date_extraction_back():
                aisis_Frame.destroy()
                date_extraction_page()
                
            # COPY PASTE CLASS SCHEDULE SCRAPER CODE IN THIS FUNCTION   
            def class_schedule_scraper(username_class_schedule, password_class_schedule):
                sched_df = weekly_sched_extract(username_class_schedule, password_class_schedule)
                sheets_main(sched_df, '1HIzSiip1TT7wwGRrLnxLwEnqickJrbc-EunKEmY-z_k', 'Weekly Schedule')
                print("Please feel free to recheck your schedule's details. You may also customize each class's COLOR ID and DESCRIPTION")
                print('Please do not forget to input the STARTING DATES and ENDING DATES.\n')
            
            # start_scraper saves the inputs from tkinter and plugs in the class_schedule_scraper function, placed below because the class_schedule_scraper has to be defined first
            def start_scraper():
                messagebox.showinfo(title="Login Validation", message="Once you press OK, the scraper will open AISIS. Please wait patiently for the scraper.")
                username_class_schedule = username_entry.get()
                password_class_schedule = password_entry.get()
                threading.Thread(target=class_schedule_scraper, args=(username_class_schedule, password_class_schedule)).start() # this calls the class_schedule_scraper function with the appropriate args
            
            # x------------------------------------------------------------------------------------------------------------------------------------x #
            # everything below this but within the def aisis_credentials page are used for aisis credentials page generator
            aisis_Frame = Frame(tk_root, bg="#402E7A")
            aisis_Frame.pack()
            
            # widgets variables
            login_label = Label(aisis_Frame, text="AISIS Credentials", bg="#402E7A", fg="#FFFFFF", font=("Arial",30))
            username_label = Label(aisis_Frame, text="Username", bg="#402E7A", fg="#FFFFFF", font=("Arial",16))
            username_entry = Entry(aisis_Frame)
            password_label = Label(aisis_Frame, text="Password", bg="#402E7A", fg="#FFFFFF", font=("Arial",16))
            password_entry = Entry(aisis_Frame, show="*")
            login_button = Button(aisis_Frame, text="Enter", font=("Arial",16), command = start_scraper)
            aisis_back_button = Button(aisis_Frame,text="BACK", command = aisis_to_date_extraction_back)
    
            # widgets placer
            login_label.grid(row=0, column=0, columnspan=2, sticky="news", pady=40)
            username_label.grid(row=1, column=0)
            username_entry.grid(row=1, column=1, pady=20)
            password_label.grid(row=2, column=0)
            password_entry.grid(row=2, column=1, pady=20)
            login_button.grid(row=3, column=0, columnspan=2, pady=30)
            aisis_back_button.grid(row=4, column=0, columnspan=2)
            
        # x------------------------------------------------------------------------------------------------------------------------------------x #
        # everything under the def aisis_credentials_class_schedule_page is used to generate the credentials page for the SYLLABUS DATES SCRAPER
        def aisis_credentials_syllabus_dates_page():
            
            # removes the container of the date_extraction page, to be replaced by the aisis credentials
            date_extraction_Frame.destroy()
            
            # when called, destroys aisis_credentials_syllabus_dates_page recalls the date_extraction_page
            def aisis_to_date_extraction_back():
                aisis_Frame.destroy()
                date_extraction_page()
                
            # COPY PASTE SYLLABUS DATES SCRAPER CODE IN THIS FUNCTION    
            def syllabus_dates_scraper(username_syllabus_dates, password_syllabus_dates):
                driver = webdriver.Chrome()
                link = r"https://aisis.ateneo.edu/j_aisis/J_VMCS.do"
    
                driver.get(link)
                time.sleep(1)
                
                driver.find_element(By.NAME, 'userName').send_keys(username_syllabus_dates)
                driver.find_element(By.NAME, 'password').send_keys(password_syllabus_dates)
                
                driver.find_element(By.NAME, "submit").click()
                time.sleep(1)
                driver.find_element(By.XPATH, '//a[contains(@href, "J_VCEC.do")]').click()
               
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                time.sleep(1.5)
                              
                pdf_links = []
                
                # Assuming the PDF links are in <a> tags and have an href ending in .pdf
                for a_tag in soup.find_all('a', href=True):
                    pdf_link = a_tag['href']
                    if pdf_link.endswith('.pdf'):
                        pdf_links.append(pdf_link)
                
                driver.quit()

                client_df = pd.DataFrame(columns = ['Subject', 'Summary', 'Location', 'Date'])
                client_df.drop(client_df.index,inplace=True) 
                total_time = 0
                                
                for pdf_url in pdf_links:
                    if not os.path.exists('pdfs'):
                        os.makedirs('pdfs')
                    
                    response = requests.get(pdf_url)
                    pdf_name = os.path.join('pdfs', pdf_url.split('/')[-1])
                
                    # Check if the PDF already exists
                    if os.path.exists(pdf_name):
                        print(f"File already exists: {pdf_name}. Skipping download.")
                    else:   
                        with open(pdf_name, 'wb') as pdf_file:
                            pdf_file.write(response.content)
                        
                        print(f'Downloaded PDF: {pdf_name}')
                
                    text = ""
                    with fitz.open(pdf_name) as pdf_document:
                        for page_num in range(pdf_document.page_count):
                            page = pdf_document[page_num]
                            text += page.get_text()  
                    
                    print('Please wait for processing...')
                    
                    start_time = time.time()  # Record the start time
                    
                    word_list = word_list_creation(text)
                    date_dict = date_find(word_list, text)
                    panda_creation(date_dict, word_list, client_df)
                      
                    end_time = time.time() # Record the end time
                    
                    pdf_time = round(end_time - start_time, 2)
                    
                    print(f'File {pdf_links.index(pdf_url)+1} of {len(pdf_links)} processed! Total Time Taken: {pdf_time} seconds\n')
                    
                    total_time += pdf_time
                    
                print(f'Processing Finished! It took {round(total_time, 2)} seconds to process the {len(pdf_links)} documents!\n')

                # This Function copy pastes everything from the pandas DataFrame into a gsheets document
                sheets_main(client_df, '1m8badixRPCpTt-dKThNeqW98PK4naJpHsAoUKA_0Uro', 'Syllabus Dates')    
                print("Please feel free to customize your events' details. You may also customize each event's COLOR ID and DESCRIPTION\n")

            # start_scraper saves the inputs from tkinter and plugs in the class_schedule_scraper function, placed below because the syllabus_dates_scraper has to be defined first
            def start_scraper():
                messagebox.showinfo(title="Login Validation", message="Once you press OK, the scraper will open AISIS. Please wait patiently for the scraper.")
                username_syllabus_dates = username_entry.get()
                password_syllabus_dates = password_entry.get()
                threading.Thread(target=syllabus_dates_scraper, args=(username_syllabus_dates, password_syllabus_dates)).start() # this calls the syllabus_dates_scraper function with the appropriate args
                  
            # everything below this but within the def aisis_credentials page are used for aisis credentials page generator
            aisis_Frame = Frame(tk_root, bg="#402E7A")
            aisis_Frame.pack()
            
            # widgets variables
            login_label = Label(aisis_Frame, text="AISIS Credentials", bg="#402E7A", fg="#FFFFFF", font=("Arial",30))
            username_label = Label(aisis_Frame, text="Username", bg="#402E7A", fg="#FFFFFF", font=("Arial",16))
            username_entry = Entry(aisis_Frame)
            password_label = Label(aisis_Frame, text="Password", bg="#402E7A", fg="#FFFFFF", font=("Arial",16))
            password_entry = Entry(aisis_Frame, show="*")
            login_button = Button(aisis_Frame, text="Enter", font=("Arial",16), command = start_scraper)
            aisis_back_button = Button(aisis_Frame,text="BACK", command = aisis_to_date_extraction_back)
    
            # widgets placer
            login_label.grid(row=0, column=0, columnspan=2, sticky="news", pady=40)
            username_label.grid(row=1, column=0)
            username_entry.grid(row=1, column=1, pady=20)
            password_label.grid(row=2, column=0)
            password_entry.grid(row=2, column=1, pady=20)
            login_button.grid(row=3, column=0, columnspan=2, pady=30)
            aisis_back_button.grid(row=4, column=0, columnspan=2)
        # x------------------------------------------------------------------------------------------------------------------------------------x #
        def finalize_dates():
            messagebox.showinfo(title="WARNING", message="Before clicking OK, make sure to have scraped both of the dates from the syllabus and the class schedule from AISIS.")

            create_events()
        
        
            # *****EXTEND THIS CODE WHEN THE """FINALIZE DATES""" CODE HAS BEEN FINISHED*****

        # x------------------------------------------------------------------------------------------------------------------------------------x #
        # date_extraction_page container
        date_extraction_Frame = Frame(tk_root, bg="#402E7A")
        date_extraction_Frame.pack()

        # title
        date_extraction_title_page = Label(date_extraction_Frame, text="Date Extraction", bg="#402E7A", fg="#FFFFFF", pady=30, font=("Arial", 30))
        date_extraction_title_page.grid(row=0, column=0, columnspan=3)
        date_extraction_title_page.configure(anchor="center")

        # the 4 buttons: "Scrape Class Schedule", "Scrape Syllabus Dates", "Finalize Dates", "BACK"
        scrape_class_schedule_button = Button(date_extraction_Frame, text="Scrape Class Schedule", command=aisis_credentials_class_schedule_page, font=("Arial", 16), bg="#4B70F5", fg="#FFFFFF")
        scrape_class_schedule_button.grid(row=1, column=0, padx=10, pady=20)
    
        scrape_syllabus_dates_button = Button(date_extraction_Frame, text="Scrape Syllabus Dates", command=aisis_credentials_syllabus_dates_page, font=("Arial", 16), bg="#4B70F5", fg="#FFFFFF")
        scrape_syllabus_dates_button.grid(row=1, column=1, padx=10, pady=20)
    
        calendar_manager_button = Button(date_extraction_Frame, text="Finalize Dates", command=finalize_dates, font=("Arial", 16), bg="#4B70F5", fg="#FFFFFF")
        calendar_manager_button.grid(row=1, column=2, padx=10, pady=20)

        date_extractor_back_button = Button(date_extraction_Frame, text="BACK", command=date_extraction_to_main_back, font=("Arial", 16), bg="#4B70F5", fg="#FFFFFF")
        date_extractor_back_button.grid(row=2, column=1, padx=10, pady=20)
        
    # x------------------------------------------------------------------------------------------------------------------------------------x #
    # for the ClassUp feature
    # Note: this is very limited
    # 1. The browser has to be Google Chrome
    # 2. The program assumes that you're logged in to Google Calendar (Selenium Log In gets blocked)
    # 3. The screenshot mechanism takes a screenshot of the entire screen and just crops out the calendar part, so if there's something blocking it, then that's the one that's gonna be screenshotted
    # 4. The background image has to be downloaded also
    def calendar_generator_page():
        # x------------------------------------------------------------------------------------------------------------------------------------x #
        # Screenshot Mechanism
        url = "https://calendar.google.com"
        webbrowser.open(url, new=2, autoraise=True)
        
        # waits for the screen to fully load
        time.sleep(5) 
        
        # Take a screenshot of the entire screen
        screenshot = pyautogui.screenshot()
        screenshot.save("full_calendar_image.png")
        
        # crop area
        crop_area = (300, 210, 1850, 1000)  # make sure that the google chrome is in full screen
        
        # crops the screenshot
        full_calendar_image = Image.open("full_calendar_image.png").convert("RGBA")
        cropped_calendar_image = full_calendar_image.crop(crop_area)
        cropped_calendar_image.save("cropped_calendar_image.png")
        ## x------------------------------------------------------------------------------------------------------------------------------------x #
        # Image Manipulation
        
        # Image Processing 1: makes the foreground transparent before overlaying
        image = Image.open("cropped_calendar_image.png").convert("RGBA")
        pixels = image.getdata()
        new_pixels = []
        for item in pixels:
            if item[:3] == (255,255,255):
                new_pixels.append((255,255,255,0))
            else:
                new_pixels.append(item)
        image.putdata(new_pixels)
        image.save("cropped_calendar_image_transparent.png")
        
        # Image Processing 2 and 3: enhances the images; using the transparent foreground, overlay it with the background
        foreground_image = Image.open("cropped_calendar_image_transparent.png").convert("RGBA")
        try:
            background_image = Image.open("background_image.jpg").convert("RGBA") # assuming that the background_image is saved as "background_image.jpg"
        except FileNotFoundError:
            print("Make sure that the background image is downloaded and is saved as 'background_image.jpg'")
        except Exception:
            print("An error occurred.")
        else:
            background_image = background_image.resize(foreground_image.size, Image.LANCZOS)
        
        # Image Enhancer
        background_enhancer = ImageEnhance.Sharpness(background_image)
        background_image = background_enhancer.enhance(3)
        foreground_enhancer = ImageEnhance.Sharpness(foreground_image)
        foreground_image = foreground_enhancer.enhance(3)
        
        blended_image = Image.alpha_composite(background_image, foreground_image)
        blended_image.save("final_calendar.png")
        
    # x------------------------------------------------------------------------------------------------------------------------------------x #
    # for the show N events
    def future_events_page():

        # destroys the main page in order to show this page
        main_Frame.destroy()

        def show_future_events(next_events_number):
            get_n_events(next_events_number)

        def start_future_events():
            next_events_number = future_events_number_entry.get()
            # you can also convert the data type of next_events_number according to what the code needs, just put it next line
            threading.Thread(target=show_future_events, args=(next_events_number,)).start()
            # NOTE: args is formatted like that because threading.Thread wants the args to be wrapped in a tuple

        # back button
        def future_events_to_main_back():
            future_events_Frame.destroy()
            main_page()
            
        # container of future_events_page frame
        future_events_Frame = Frame(tk_root, bg="#402E7A")
        future_events_Frame.pack()

        # widgets
        future_events_title = Label(future_events_Frame, text="Future Events", bg="#402E7A", fg="#FFFFFF", font=("Arial",30))
        future_events_number_label = Label(future_events_Frame, text="Enter number of events you want to see next: ", bg="#402E7A", fg="#FFFFFF", font=("Arial",16))
        future_events_number_entry = Entry(future_events_Frame)
        future_events_enter_button = Button(future_events_Frame, text="Enter", font=("Arial",16), command = start_future_events)
        future_events_back_button = Button(future_events_Frame,text="BACK", command = future_events_to_main_back)

        # widgets placer
        future_events_title.grid(row=0, column=0, columnspan=2, sticky="news", pady=40)
        future_events_number_label.grid(row=1, column=0)
        future_events_number_entry.grid(row=1, column=1, pady=20)
        future_events_enter_button.grid(row=2, column=0, columnspan=2, pady=30)
        future_events_back_button.grid(row=3, column=0, columnspan=2)
        
    # x------------------------------------------------------------------------------------------------------------------------------------x #
    # insert ring chart
    def today_events_page():
        main_Frame.destroy()
        
        from google.oauth2.credentials import Credentials
        from googleapiclient.discovery import build
        import pytz
        import datetime as dt
        from collections import Counter
        import matplotlib.pyplot as plt
        from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

        def get_events_today():
            # credentials to access google calendar
            SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']  # Restore the .readonly scope
            TIMEZONE = 'Asia/Singapore'  # GMT+08:00

            creds = None
            if os.path.exists('token.json'):
                creds = Credentials.from_authorized_user_file('token.json', SCOPES)
            if not creds or not creds.valid:
                if creds and creds.expired and creds.refresh_token:
                    creds.refresh(Request())
                else:
                    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
                    creds = flow.run_local_server(port=0)
                with open('token.json', 'w') as token:
                    token.write(creds.to_json())
        
            service = build('calendar', 'v3', credentials=creds)
        
            tz = pytz.timezone(TIMEZONE)  # converts the time to Singapore Time
            now = dt.datetime.now(tz).isoformat()
            end_of_day = (dt.datetime.now(tz) + dt.timedelta(days=1)).replace(
                hour=0, minute=0, second=0, microsecond=0).isoformat()
        
            # gets the list of the events on the day
            events_result = service.events().list(calendarId='primary', timeMin=now, timeMax=end_of_day,
                                                  maxResults=100, singleEvents=True,
                                                  orderBy='startTime').execute()
            events = events_result.get('items', [])
            return events
        
        def ring_chart_schedule(event_counts):
            labels = list(event_counts.keys())  # list of name of events
            sizes = list(event_counts.values())  # list of the sizes of events compared to the day
            colors = plt.cm.Paired(range(len(labels)))  # just uses matplotlib colors depending on how many events there are
        
            fig, ax = plt.subplots()
            ax.pie(sizes, labels=labels, colors=colors, startangle=90, wedgeprops=dict(width=0.3))
        
            centre_circle = plt.Circle((0, 0), 0.70, fc='white')
            fig.gca().add_artist(centre_circle)
        
            ax.axis('equal')
            plt.title("Distribution of Today's Events")
            return fig
        
        def update_chart():
            events = get_events_today()
            event_counts = Counter(event['summary'] for event in events)
            fig = ring_chart_schedule(event_counts)
        
            for widget in today_events_Frame.winfo_children():  # when a new chart is created, it destroys all the old ones first before placing the new one
                widget.destroy()
        
            canvas = FigureCanvasTkAgg(fig, master=today_events_Frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=BOTH, expand=True)
        
        # container of the chart
        today_events_Frame = Frame(tk_root)
        today_events_Frame.pack(fill=BOTH, expand=True)
        
        # Call the update_chart function to initialize the chart
        update_chart()
    # x------------------------------------------------------------------------------------------------------------------------------------x #
        
    # main_page container
    main_Frame = Frame(tk_root, bg="#402E7A")
    main_Frame.pack()

    # title
    title_page = Label(main_Frame, text="Calendar Organizer Dashboard", bg="#402E7A", fg="#FFFFFF", pady=30, font=("Arial", 30))
    title_page.grid(row=0, column=0, columnspan=3)
    title_page.configure(anchor="center")
    
    # the 3 buttons in the main page: "Date Extraction", "Calendar Generator", "Calendar Manager"
    date_extraction_button = Button(main_Frame, text="Date Extraction", command=date_extraction_page, font=("Arial", 16), bg="#4B70F5", fg="#FFFFFF")
    date_extraction_button.grid(row=1, column=0, padx=10, pady=20)

    calendar_generator_button = Button(main_Frame, text="Calendar Generator", command=calendar_generator_page, font=("Arial", 16), bg="#4B70F5", fg="#FFFFFF")
    calendar_generator_button.grid(row=1, column=1, padx=10, pady=20)

    future_events_button = Button(main_Frame, text="Future Events", command=future_events_page, font=("Arial", 16), bg="#4B70F5", fg="#FFFFFF")
    future_events_button.grid(row=1, column=2, padx=10, pady=20)
    
    # formatting and showing the date today
    today = date.today()
    show_day_today = Label(main_Frame, text="Today is: " + today.strftime('%B %d, %Y') + ". Click button to see today's tasks: ", bg="#402E7A", fg="#FFFFFF", pady=30, font=("Arial", 20))
    show_day_today.grid(row=2, column=0, columnspan=3)
    show_day_today.configure(anchor="center")

    # the 4th button in the main page: "Today's Events"
    today_events_button = Button(main_Frame, text="Today's Events", command=today_events_page, font=("Arial", 16), bg="#4B70F5", fg="#FFFFFF")
    today_events_button.grid(row=3, column=1, padx=10, pady=20)
    
# x------------------------------------------------------------------------------------------------------------------------------------x #
# calls the main_page as the beginning
main_page()
tk_root.mainloop()